# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\iiats\Anaconda3\envs\Python37\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
### P-86で使用する
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [2]:
df_customer_1 = pd.merge(df_customer[['customer_id', 'postal_cd']],
                         df_geocode[['postal_cd', 'longitude' ,'latitude']],
                         how='inner', on='postal_cd')
df_customer_1 = df_customer_1.groupby('customer_id'). \
    agg({'longitude':'mean', 'latitude':'mean'}).reset_index(). \
    rename(columns={'longitude':'m_longitude', 'latitude':'m_latitude'})

df_customer_1 = pd.merge(df_customer, df_customer_1, how='inner', on='customer_id')
df_customer_1.head(3)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,m_longitude,m_latitude
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.31779,35.41358
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.83502,35.67193
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,139.68965,35.67374


In [16]:
# pd の DataFrame の表示カラム数を増やす
pd.set_option('display.max_columns',90)

---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [17]:
# まず公式解凍の解釈
# 距離を計算する関数を定義
def calc_distance(x1, y1, x2, y2):
    distance = 6371 * math.acos(math.sin(math.radians(y1)) * math.sin(math.radians(y2)) 
                       + math.cos(math.radians(y1)) * math.cos(math.radians(y2)) 
                            * math.cos(math.radians(x1) - math.radians(x2)))
    return distance

In [18]:
# df_customer_1 と df_store のマージ
df_tmp = pd.merge(df_customer_1, df_store, how='inner', left_on='application_store_cd', right_on='store_cd')    

In [19]:
df_tmp.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address_x,application_store_cd,application_date,status_cd,m_longitude,m_latitude,store_cd,store_name,prefecture_cd,prefecture,address_y,address_kana,tel_no,longitude,latitude,floor_area
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.31779,35.41358,S14021,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0
1,CS021313000025,砂川 あさみ,1,女性,1981-10-05,37,259-1131,神奈川県伊勢原市伊勢原**********,S14021,20150326,0-00000000-0,139.31374,35.39748,S14021,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0
2,CS021411000096,布施 杏,1,女性,1971-10-30,47,259-1114,神奈川県伊勢原市高森**********,S14021,20160621,0-00000000-0,139.32489,35.42174,S14021,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0
3,CS021415000150,早美 紗季,1,女性,1969-11-26,49,259-1141,神奈川県伊勢原市上粕屋**********,S14021,20150601,0-00000000-0,139.28880,35.41649,S14021,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0
4,CS021313000046,会田 薫,9,不明,1983-01-19,36,259-1144,神奈川県伊勢原市池端**********,S14021,20150722,0-00000000-0,139.32483,35.39936,S14021,伊勢原店,14,神奈川県,神奈川県伊勢原市伊勢原四丁目,カナガワケンイセハラシイセハラヨンチョウメ,046-123-4035,139.3129,35.40169,962.0


In [20]:
# 経度と緯度から距離を算出して、新しいカラムを作成
df_tmp['distance'] =   df_tmp[['m_longitude', 'm_latitude','longitude', 'latitude']]. \
                                apply(lambda x: calc_distance(x[0], x[1], x[2], x[3]), axis=1)

In [21]:
df_tmp[['customer_id', 'address_x', 'address_y', 'distance']].head(10)

,customer_id,address_x,address_y,distance
0,CS021313000114,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.394409
1,CS021313000025,神奈川県伊勢原市伊勢原**********,神奈川県伊勢原市伊勢原四丁目,0.474282
2,CS021411000096,神奈川県伊勢原市高森**********,神奈川県伊勢原市伊勢原四丁目,2.480155
3,CS021415000150,神奈川県伊勢原市上粕屋**********,神奈川県伊勢原市伊勢原四丁目,2.734723
4,CS021313000046,神奈川県伊勢原市池端**********,神奈川県伊勢原市伊勢原四丁目,1.111911
5,CS021103000002,神奈川県伊勢原市西富岡**********,神奈川県伊勢原市伊勢原四丁目,2.384941
6,CS021214000028,神奈川県伊勢原市桜台**********,神奈川県伊勢原市伊勢原四丁目,1.399344
7,CS021512000095,神奈川県伊勢原市沼目**********,神奈川県伊勢原市伊勢原四丁目,1.993991
8,CS021613000002,神奈川県伊勢原市桜台**********,神奈川県伊勢原市伊勢原四丁目,1.399344
9,CS021412000147,神奈川県伊勢原市三ノ宮**********,神奈川県伊勢原市伊勢原四丁目,3.507680


In [22]:
# 以前に書いていたコードを利用
# ２地点間の距離
def sphere_dist(x1_lat, x1_lon, x2_lat, x2_lon):

    # Define earth radius (km)
    R_earth = 6371
    # Convert degrees to radians
    x1_lat, x1_lon, x2_lat, x2_lon = map(np.radians, [x1_lat, x1_lon, x2_lat, x2_lon])
    # Compute distances along lat, lon dimensions
    dlat = x2_lat - x1_lat
    dlon = x2_lon - x1_lon
    # Compute haversine distance
    a = np.sin(dlat/2.0)**2 + np.cos(x1_lat) * np.cos(x2_lat) * np.sin(dlon/2.0)**2
    
    return 2 * R_earth * np.arcsin(np.sqrt(a))

In [23]:
# df_customer_1 と df_store のマージ
df_tmp2 = pd.merge(df_customer_1, df_store, how='inner', left_on='application_store_cd', right_on='store_cd')    

In [24]:
# 経度と緯度から距離を算出して、新しいカラムを作成
df_tmp2['distance'] =   df_tmp2[['longitude', 'latitude', 'm_longitude', 'm_latitude',]]. \
                                apply(lambda x: sphere_dist(x[0], x[1], x[2], x[3]), axis=1)

In [25]:
df_tmp2[['customer_id', 'address_x', 'address_y', 'distance']].head(10)

,customer_id,address_x,address_y,distance
0,CS021313000114,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.140524
1,CS021313000025,神奈川県伊勢原市伊勢原**********,神奈川県伊勢原市伊勢原四丁目,0.367060
2,CS021411000096,神奈川県伊勢原市高森**********,神奈川県伊勢原市伊勢原四丁目,2.153135
3,CS021415000150,神奈川県伊勢原市上粕屋**********,神奈川県伊勢原市伊勢原四丁目,2.956009
4,CS021313000046,神奈川県伊勢原市池端**********,神奈川県伊勢原市伊勢原四丁目,1.341027
5,CS021103000002,神奈川県伊勢原市西富岡**********,神奈川県伊勢原市伊勢原四丁目,2.099841
6,CS021214000028,神奈川県伊勢原市桜台**********,神奈川県伊勢原市伊勢原四丁目,1.115342
7,CS021512000095,神奈川県伊勢原市沼目**********,神奈川県伊勢原市伊勢原四丁目,2.318651
8,CS021613000002,神奈川県伊勢原市桜台**********,神奈川県伊勢原市伊勢原四丁目,1.115342
9,CS021412000147,神奈川県伊勢原市三ノ宮**********,神奈川県伊勢原市伊勢原四丁目,4.295739


---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [27]:
# 公式解答の解釈
# まずdf_receipt を groupe()
df_tmp = df_receipt.groupby('customer_id').agg({'amount':sum}).reset_index()
df_tmp.head(3)

,customer_id,amount
0,CS001113000004,1298
1,CS001114000005,626
2,CS001115000010,3044


In [28]:
# dr_customer と df_tmp を customer_id をキーとしてマージ
df_customer_u = pd.merge(df_customer, df_tmp, how='left', on='customer_id').sort_values(['amount', 'customer_id']
                                                                                        , ascending=[False, True])
df_customer_u.head(3)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,amount
16905,CS017415000097,福士 千夏,1,女性,1973-04-03,45,166-0014,東京都杉並区松ノ木**********,S13017,20151209,F-20101006-F,23086.0
12692,CS015415000185,岩淵 はるみ,1,女性,1973-09-19,45,135-0043,東京都江東区塩浜**********,S13015,20150322,F-20101014-F,20153.0
13550,CS031414000051,長澤 沙知絵,1,女性,1973-04-25,45,151-0064,東京都渋谷区上原**********,S13031,20150823,F-20101009-F,19202.0


In [29]:
# pandas.DataFrame, Seriesの重複した行を削除
# https://note.nkmk.me/python-pandas-duplicated-drop-duplicates/
df_customer_u.drop_duplicates(subset=['customer_name', 'postal_cd'], keep='first', inplace=True)
print('減少数: ', len(df_customer) - len(df_customer_u))

減少数:  30


---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [31]:
# 公式解答の解釈
# 前問の df_customer_u と df_customer を customer_id をキーとしてマージ
df_customer_n = pd.merge(df_customer, df_customer_u[['customer_name', 'postal_cd', 'customer_id']],
                        how='inner', on =['customer_name', 'postal_cd'])
df_customer_n.head(3)

,customer_id_x,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,customer_id_y
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,CS021313000114
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,CS037613000071
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,CS031415000172


In [32]:
# カラム名をリネーム
df_customer_n.rename(columns={'customer_id_x':'customer_id', 'customer_id_y':'integration_id'}, inplace=True)

In [34]:
# pandasでユニークな要素の個数、頻度（出現回数）をカウント
# https://note.nkmk.me/python-pandas-value-counts/
print('ID数の差', len(df_customer_n['customer_id'].unique()) - len(df_customer_n['integration_id'].unique()))

ID数の差 30


---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

In [35]:
del df_customer_1
del df_customer_n

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [38]:
# 公式解答
df_tmp = pd.merge(df_customer, df_receipt['customer_id'], how='inner', on='customer_id')
df_train, df_test = train_test_split(df_tmp, test_size=0.2, random_state=71)
print('学習データ割合: ', len(df_train) / len(df_tmp))
print('テストデータ割合: ', len(df_test) / len(df_tmp))

学習データ割合:  0.7999908650771901
テストデータ割合:  0.2000091349228099


In [45]:
# AAMLP(Approaching (Almost) Any Macine Learning Problems) で紹介されている方法

# df_tmp をランダムに並べ替える
df_tmp = df_tmp.sample(frac = 1).reset_index(drop = True)

df_train = df_tmp.head(int(len(df_tmp)*0.8))
df_test = df_tmp.tail(int(len(df_tmp)*0.2))

print('学習データ割合: ', len(df_train) / len(df_tmp))
print('テストデータ割合: ', len(df_test) / len(df_tmp))

学習データ割合:  0.7941176470588235
テストデータ割合:  0.17647058823529413


---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [46]:
# 難しかったので公式解答のままです。
df_tmp = df_receipt[['sales_ymd', 'amount']].copy() # deep copy: 値渡し
df_tmp['sales_ym'] = df_tmp['sales_ymd'].astype('str').str[0:6]
df_tmp = df_tmp.groupby('sales_ym').agg({'amount':'sum'}).reset_index()
df_tmp.head(3)

,sales_ym,amount
0,201701,902056
1,201702,764413
2,201703,962945


In [47]:
# 関数化することで長期間データに対する多数のデータセットもループなどで処理できるようにする --> 自分では書けない！
def split_data(df, train_size, test_size, slide_window, start_point):
    train_start = start_point * slide_window
    test_start = train_start + train_size
    return df[train_start : test_start], df[test_start : test_start + test_size]

In [48]:
df_train_1, df_test_1 = split_data(df_tmp, train_size=12, test_size=6, slide_window=6, start_point=0)
df_train_2, df_test_2 = split_data(df_tmp, train_size=12, test_size=6, slide_window=6, start_point=1)
df_train_3, df_test_3 = split_data(df_tmp, train_size=12, test_size=6, slide_window=6, start_point=2)

In [49]:
df_train_1

,sales_ym,amount
0,201701,902056
1,201702,764413
2,201703,962945
3,201704,847566
4,201705,884010
5,201706,894242
6,201707,959205
7,201708,954836
8,201709,902037
9,201710,905739


In [50]:
df_test_1

,sales_ym,amount
12,201801,944509
13,201802,864128
14,201803,946588
15,201804,937099
16,201805,1004438
17,201806,1012329
